In [1]:
import torch.nn as nn
class Decoder(nn.Module):
    def __init__(self, vocab_size, d_model, n_heads, n_layers, max_seq_len, dropout=0.1):
        super().__init__()
        self.token_emb = nn.Embedding(vocab_size, d_model)
        self.pos_emb = nn.Parameter(torch.zeros(1, max_seq_len, d_model))
        self.dropout = nn.Dropout(dropout)

        self.layers = nn.ModuleList([
            nn.ModuleDict({
                'ln1': nn.LayerNorm(d_model),
                'attn': nn.MultiheadAttention(d_model, n_heads, dropout=dropout, batch_first=True),
                'ln2': nn.LayerNorm(d_model),
                'mlp': nn.Sequential(
                    nn.Linear(d_model, 4 * d_model),
                    nn.GELU(),
                    nn.Linear(4 * d_model, d_model),
                    nn.Dropout(dropout),
                )
            }) for _ in range(n_layers)
        ])

        self.ln_f = nn.LayerNorm(d_model)
        self.head = nn.Linear(d_model, vocab_size, bias=False)
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, (nn.Linear, nn.Embedding)):
            nn.init.normal_(module.weight, mean=0, std=0.02)
        if isinstance(module, nn.LayerNorm) and module.bias is not None:
            nn.init.zeros_(module.bias)

    def forward(self, x):
        B, T = x.size()
        x = self.token_emb(x) + self.pos_emb[:,:T]
        x = self.dropout(x)
        attn_mask = torch.triu(torch.ones(T, T, device=x.device), diagonal=1).bool()

        for layer in self.layers:
            x_norm = layer['ln1'](x)
            attn_output, _ = layer['attn'](
                x_norm, x_norm, x_norm,
                attn_mask=attn_mask,
                need_weights=False,
            )
            x = x + attn_output
            x = x + layer['mlp'](layer['ln2'](x))

        x = self.ln_f(x)
        logits = self.head(x)
        return logits

In [2]:
from accelerate import Accelerator
from torch.optim import AdamW
from tqdm import tqdm
from bitsandbytes import optim as bnb_optim

def train(model, dataloader, vocab_size, epochs=3, lr=3e-4, weight_decay=0.0, early_stop_loss=0.1):
    accelerator = Accelerator()
    device = accelerator.device

    model.to(device)
    optimizer = bnb_optim.AdamW8bit(model.parameters(), lr=lr, weight_decay=weight_decay)
    criterion = nn.CrossEntropyLoss(ignore_index=0)

    model, optimizer, dataloader = accelerator.prepare(model, optimizer, dataloader)
    model.train()
    for epoch in range(epochs):
        pbar = tqdm(dataloader, desc=f"Epoch: {epoch+1}", disable=not accelerator.is_local_main_process)
        for batch in pbar:
            input_ids = batch[:, :-1]
            labels = batch[:, 1:]

            outputs = model(input_ids)
            #loss = criterion(outputs.view(-1, vocab_size), labels.view(-1))
            loss = criterion(outputs.reshape(-1, vocab_size), labels.reshape(-1))


            optimizer.zero_grad()
            accelerator.backward(loss)
            optimizer.step()

            pbar.set_postfix(loss=loss.item())

            if loss.item() < early_stop_loss:
                print(f"Training stopped early at epoch {epoch+1}, batch {pbar.n} due to train_loss < {early_stop_loss}")
                return  # 訓練を中止

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from src.notebook.dataloader import JPNDataset

dataset = torch.load("../data/dataset_1024.pt", weights_only=False)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

model = Decoder(vocab_size=22217, d_model=768, n_heads=12, n_layers=12, max_seq_len=1024, dropout=0.1)
train(model, dataloader, vocab_size=22217, epochs=3, lr=3e-4, weight_decay=0.01)

Epoch: 1:   0%|          | 37/148034 [23:25<1772:26:15, 43.11s/it, loss=6]   

In [4]:
torch.save(model.state_dict(), "../data/model.pth")

In [25]:
import torch
import json
import src.notebook.path as path
from src.notebook.charTokenizer import CharTokenizer

model = Decoder(vocab_size=22217, d_model=768, n_heads=12, n_layers=12, max_seq_len=1024)
model.load_state_dict(torch.load("../data/model.pth"))
model.eval()

tokenizer = CharTokenizer()
with open(path.charTokenizer, 'r', encoding='utf-8') as f:
    tokenizer.vocab = json.load(f)
inputs = tokenizer(
    "荒れた内を避ける為か中間付近までは馬場の中央付近を走行。道中ペースを緩め脚をためると、そのまま直線も先頭でゴールした。逃げての上がりは32.9で上がり最速タイ。 武豊騎手は「ポンと出て、無理に引っ張ることもなく、マイペースで行けた。ラストでひと伸びして能力の高さを感じた」とコメント。ききょうステークス以来実に1年ぶりの勝利を飾り、素質の高さを見せた。続く逆瀬川ステークスでは前走の走りを評価されてか、古馬と同じ55kgの斤量を課された。チャンピオンズカップの裏開催であった為、武豊騎手から吉田隼人騎手に乗り替わり、朝日杯FS以来のコンビ結成となった。スムーズにゲートを出ると、そのまま内3番手を追走。最後直線は力強く抜け出して、2連勝でのOP入りを決めた。吉田隼人騎手は「約1年ぶりに乗せていただきましたが成長しています。出たなりでいい位置をキープできました。抜け出してから、左にもたれる癖はあるが、上がり勝負にも対応してくれました。競馬に幅が広がったし、これからが楽しみです」と振り返った。2023年（4歳）.明け4歳の始動戦に選ばれたのは東京芝2000mのリステッド戦である[白富士ステークスとされた。当日は前走ローズステークス2着と好走したサリエラに次ぐ2番人気に評価された。レースはドーブネが最内枠から好スタートでハナを奪い、武豊のエスコートで1000m59.9秒という絶妙な時計で逃げを打つ。直線に入り粘りの逃げで後続を離すかに思えたが、残り200mあたりから失速。最後は後方から末脚を伸ばしてきたサリエラに交わされた。それでも内を通って伸びてきていたヤマニンサルバムには抜かせず2着を確保した。レース後武豊は「楽なペースだったけどね…。1ハロンぐらい少し距離が長いのかな」とコメントを残し、2000ｍはドーブネにとって距離が長い可能性が示唆された。"
    , max_length=1024, return_tensors="pt")
with torch.no_grad():
    outputs = model(inputs["input_ids"])
    predictions = torch.argmax(outputs, dim=-1)
output_text = tokenizer.decode(predictions[0])
print(output_text)


荒れた内をたける為か中間付近までは馬場の中央付近を走行。道中ペースをけめ脚をためると、そのまま直線も先頭でゴールした。逃げての上がりは32.9で上がり最速タイ。 武豊騎手は「ポンと出て、無理に引っ張ることもなく、マイペースで行けた。ラストでひと伸びして能力の高さを感じた」とコメント。ききょうステークス以来実に1年ぶりの勝利を飾り、素質の高さを見せた。続く逆瀬川ステークスでは前走の走りを評価されてか、古馬と同じ55kgの滑量を課された。チャンピオンズカップの裏開催であった為、武豊騎手から吉田+人騎手に乗り替わり、朝日杯FS以来のコンビ結成となった。スムーズにゲートを出ると、そのまま内3番手を追走。最後直線は力強く抜け出して、2連勝でのOP入りを決めた。吉田+人騎手は「約1年ぶりに乗せていただきましたが成長しています。出たなりでいい位置をキープできました。抜け出してから、左にもたれる布はあるが、上がり勝負にも対応してくれました。競馬に幅が広がったし、これからが楽しみです」と振り返った。2023年（4歳）.明け4歳の始動戦に選ばれたのは東京芝2000mのリステッド戦である[白富士ステークスとされた。当日は前走ローズステークス2着と好走したサリエラに次ぐ2番人気に評価された。レースはドーブネが最内枠から好スタートでハナを奪い、武豊のエスコートで1000m59.9秒という絶準な時計で逃げを打つ。直線に入り括りの逃げで後続を離すかに思えたが、残り200mあたりから失速。最後は後方から末脚を伸ばしてきたサリエラに交わされた。それでも内を通って伸びてきていたヤマニンサルバムには抜かせず2着を確保した。レース後武豊は「楽なペースだったけどね…。1ハロンぐらい少し距離が長いのかな」とコメントを残し、2000『はドーブネにとって距離が長い可能性が示唆された。顔顔顔替顔顔顔顔顔顔誉顔顔緩顔盤誉盤顔誉顔概顔替誉概概顔概盤概替概概概盤概誉顔概概概概概概概概概概概概概誉顔概概誉概概概概概概概概概概概概盤概概概概概概概概概概概盤概概概概概誉概概概概概概概概概概概概概概概概概概概概概概概概概概概概概概概概概概概概概概赤概概概概概概概概概概概概概概概概概概概概概概概概概概概概概概概概概概概概概概概概概概概概概概概概概赤概概概概概概概概概概概概赤赤概概概系概概概概概概概概概系概概概概)概概概概)概赤系系